<a href="https://colab.research.google.com/github/WeiShi324/WeiShi324/blob/main/BFL_Neronetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Stock_Screening.xlsx to Stock_Screening.xlsx


In [2]:
!pip install tensorflow

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
file_path = 'Stock_Screening.xlsx'

In [4]:
def load_data():
    # Load target variable
    target_df = pd.read_excel(file_path, sheet_name='Y Label')
    target_df.set_index('Date', inplace=True)

    # Load explanatory variables from different tabs
    explanatory_dfs = {}
    features = ['Pt12mE', 'EVt12mEBIDA','PEG','Pt12CFPS','ROA','ROE','EBITDAMagin', 'EPSGrowth','EBITinterest','CFO1yrGrowth']  # Add all your feature tab names here

    for feature in features:
        try:
            df = pd.read_excel(file_path, sheet_name=feature)
            df.set_index('Date', inplace=True)
            explanatory_dfs[feature] = df
        except:
            print(f"Could not load sheet: {feature}")

    return target_df, explanatory_dfs

In [5]:
# Prepare the dataset
def prepare_data(target_df, explanatory_dfs):
    # Combine all features into one dataframe
    combined_features = []
    for feature_name, df in explanatory_dfs.items():
        # Reshape each feature dataframe and add feature name prefix
        for column in df.columns:
            temp_df = df[[column]].copy()
            temp_df.columns = [f"{feature_name}_{column}"]
            combined_features.append(temp_df)

    # Join all features
    X = pd.concat(combined_features, axis=1)

    # Align X and y based on dates and stocks
    common_dates = X.index.intersection(target_df.index)
    X = X.loc[common_dates]
    y = target_df.loc[common_dates]

    # Convert target to binary (assuming underperform=0, outperform=1)
    y = (y > y.mean()).astype(int)  # You might want to adjust this based on your Sharpe ratio labeling

    return X, y



In [6]:
# Build neural network model
def build_model(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    return model

In [7]:


# Main execution
def main():
    # Load data
    print("Loading data...")
    target_df, explanatory_dfs = load_data()

    # Prepare data
    print("Preparing data...")
    X, y = prepare_data(target_df, explanatory_dfs)

    # Handle missing values
    X = X.fillna(X.mean())
    y = y.fillna(y.mean())

    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42
    )

    # Build and train model
    model = build_model(X_train.shape[1])

    print("Training model...")
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        verbose=1
    )

    # Evaluate model
    print("\nEvaluating model...")
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test accuracy: {test_accuracy:.4f}")

    # Make predictions
    predictions = model.predict(X_test)
    predictions_binary = (predictions > 0.5).astype(int)

    return model, history, predictions_binary

if __name__ == "__main__":
    # Set random seed for reproducibility
    np.random.seed(42)
    tf.random.set_seed(42)

    try:
        model, history, predictions = main()

        # Optional: Save the model
        model.save('stock_selection_model.h5')
        print("Model saved successfully")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

Loading data...
An error occurred: name 'file_path' is not defined
